In [1]:
#Importing required libraries
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
eps = np.finfo(float).eps
from numpy import log2 as log

In [2]:
train = pd.read_csv(r"C:\Users\Ruchi\Documents\titanic_train.csv")
test = pd.read_csv(r"C:\Users\Ruchi\Documents\titanic_test.csv")

In [3]:
org_train = train.copy()
org_test = test.copy()

In [4]:
#Making a full dataset containing train and test data
data = train.append(test, sort = True)

In [5]:
data.head(5)

,age,boat,body,cabin,embarked,fare,home.dest,name,parch,passenger_id,pclass,sex,sibsp,survived,ticket
0,NaN,13,NaN,NaN,Q,7.7333,NaN,"Smyth, Miss. Julia",0,1216,3,female,0,1.0,335432
1,38.0,NaN,NaN,NaN,S,8.6625,Croatia,"Cacic, Mr. Luka",0,699,3,male,0,0.0,315089
2,30.0,NaN,NaN,NaN,S,24.1500,NaN,"Van Impe, Mrs. Jean Baptiste (Rosalie Paula Go...",1,1267,3,female,1,0.0,345773
3,54.0,4,NaN,NaN,S,23.0000,"Cornwall / Akron, OH","Hocking, Mrs. Elizabeth (Eliza Needs)",3,449,2,female,1,1.0,29105
4,40.0,NaN,NaN,NaN,S,13.0000,"Barre, Co Washington, VT","Veal, Mr. James",0,576,2,male,0,0.0,28221


In [6]:
#Cleaning
data.dropna()
data['sex'] =  data['sex'].map( {'female': 0, 'male': 1} ).astype(int)
#Grouping ages
data.loc[data['age'] <= 16, 'age'] = 0
data.loc[(data['age'] > 16) & (data['age'] < 50), 'age'] = 1
data.loc[data['age'] >= 50, 'age'] = 2 

## How it's mapped:
F : 0; M: 1
Young : 0; Adult: 1; Old: 2

In [7]:
def entropy(value_ratios):
    e = 0
    for value in value_ratios:
        e += -(value*log(value))
    return e

In [8]:
'''Giving erronous value
def wt_avg_e(dict_obj):
    wt_e = 0
    for i in dict_obj:
        wt_e = -(i*log(dict_obj[i]))
    print (i, dict_obj[i])
    return wt_e


wt_age_e = wt_avg_e(wt_age_dict)
print(wt_age_e)
    ''' 

'Giving erronous value\ndef wt_avg_e(dict_obj):\n    wt_e = 0\n    for i in dict_obj:\n        wt_e = -(i*log(dict_obj[i]))\n    print (i, dict_obj[i])\n    return wt_e\n\n\nwt_age_e = wt_avg_e(wt_age_dict)\nprint(wt_age_e)\n    '

In [9]:
def info_gain(parent_e, wt_avg_e):
    info_gain = 0
    info_gain = parent_e - wt_avg_e
    return info_gain

## Some universal variables used here:


In [10]:
# Number of passengers:
total_pass = data['passenger_id'].count() #1309


#Number of passengers by AGE
num_young = len(data.loc[data['age'] == 0])
num_adult = len(data.loc[data['age'] == 1])
num_old = len(data.loc[data['age'] == 2])

#Number of passengers by SEX
num_male = len(data['sex'] == 1)
num_female = len(data['sex'] == 0)

num_young_f = len(data.loc[(data['age'] == 0) & (data['sex'] == 0)])
num_young_m = len(data.loc[(data['age'] == 0) & (data['sex'] == 1)])

num_adult_f = len(data.loc[(data['age'] == 1) & (data['sex'] == 0)])
num_adult_m = len(data.loc[(data['age'] == 1) & (data['sex'] == 1)])

num_old_f = len(data.loc[(data['age'] == 2) & (data['sex'] == 0)])
num_old_m = len(data.loc[(data['age'] == 2) & (data['sex'] == 1)])


# Total passengers based on PORT OF EMBARKATION
total_q = len(data.loc[data['embarked'] == 'Q'])
total_s = len(data.loc[data['embarked'] == 'S'])
total_c = len(data.loc[data['embarked'] == 'C'])

# Port - Q
num_q_first = len(data.loc[(data['embarked'] == 'Q') & (data['pclass'] == 1)])
num_q_second = len(data.loc[(data['embarked'] == 'Q') & (data['pclass'] == 2)])
num_q_third = len(data.loc[(data['embarked'] == 'Q') & (data['pclass'] == 3)])


#Port - S

num_s_first = len(data.loc[(data['embarked'] == 'S') & (data['pclass'] == 1)])
num_s_second = len(data.loc[(data['embarked'] == 'S') & (data['pclass'] == 2)])
num_s_third = len(data.loc[(data['embarked'] == 'S') & (data['pclass'] == 3)])

#Port - C

num_c_first = len(data.loc[(data['embarked'] == 'C') & (data['pclass'] == 1)])
num_c_second = len(data.loc[(data['embarked'] == 'C') & (data['pclass'] == 2)])
num_c_third = len(data.loc[(data['embarked'] == 'C') & (data['pclass'] == 3)])


## Tree 1: Age-Sex-Survival

Age - Y,A,O - M,F
Y(m,f)
A(m,f)
O(m,f)

y_m / tot_y
a_f / tot_a


In [11]:
age_ratios = data['age'].value_counts(normalize=True)
age_e = entropy(age_ratios)



#Young sex-wise ratios
young_f_ratio = num_young_f/num_young
young_m_ratio = num_young_m/num_young

#Adult sex-wise ratios


adult_f_ratio = num_adult_f/num_adult
adult_m_ratio = num_adult_m/num_adult

#Old sex-wise ratios


old_f_ratio = num_old_f/num_old
old_m_ratio = num_old_m/num_old
                                
#Making a series object
young_ratio = pd.Series([young_f_ratio, young_m_ratio])
adult_ratio = pd.Series([adult_f_ratio, adult_m_ratio])
old_ratio = pd.Series([old_f_ratio, old_m_ratio])

#Getting conditional entropies
young_cond_e = entropy(young_ratio)  #entropies
adult_cond_e = entropy(adult_ratio)
old_cond_e = entropy(old_ratio)

#Weighted avg. entropy
ratio_y = num_young/total_pass
ratio_a = num_adult/total_pass
ratio_o = num_old/total_pass

'''ratio_age_subgroups = pd.Series([ratio_y, ratio_a, ratio_o])
wt_age_dict = {ratio_y : young_cond_e, ratio_a : adult_cond_e, ratio_o : old_cond_e}
wt_age_data = {'Subgroup Ratio': [ratio_y, ratio_a, ratio_o], 'Cond E':[young_cond_e,adult_cond_e,old_cond_e]}
wt_age_df = pd.DataFrame(wt_age_data)
wt_age_df
wt_age_series = pd.Series(wt_age_data)
wt_age_series
'''
wt_avg_age_e = (ratio_y*young_cond_e) + (ratio_a*adult_cond_e) + (ratio_o*old_cond_e)
info_gain_age = info_gain(age_e, wt_avg_age_e)
print(info_gain_age)


0.2591223932930248


## Tree 2: Sex-Age-Survival

In [12]:
sex_ratios = data['sex'].value_counts(normalize=True)
sex_e = entropy(sex_ratios)

#Male - Y, A, O
m_young_ratio = num_young_m/num_male
m_adult_ratio = num_adult_m/num_male
m_old_ratio = num_old_m/num_male

#Female - Y, A, O
f_young_ratio = num_young_f/num_female
f_adult_ratio = num_adult_f/num_female
f_old_ratio = num_old_f/num_female

#Making series objects
m_age_ratio = pd.Series([m_young_ratio, m_adult_ratio, m_old_ratio])
f_age_ratio = pd.Series([f_young_ratio, f_adult_ratio, f_old_ratio])

#Conditional entropies
m_cond_e = entropy(m_age_ratio)
f_cond_e = entropy(f_age_ratio)

#Weighted avg entropy
ratio_m = num_male / total_pass
ratio_f = num_female / total_pass

wt_avg_sex_e = (ratio_m*m_cond_e) + (ratio_f*f_cond_e)
info_gain_sex = info_gain(sex_e, wt_avg_sex_e)
print(info_gain_sex)

-0.8867576845634857


## Tree 3: Port-Class-Sex-Age
In getting the conditional entropies of Class | Port (eg. 1st class passenger embarked from port Q),
we're faced witht the question of how to get the conditional entropies for multiple attributes which are split from 
other parent attributes in a larger tree.
The question comes to play especially because the wt.avg. entropies that we'll need to take in order to get the information
gain for the tree.

Conditional entropy : the conditional entropy quantifies the amount of information needed to describe the outcome of a random variable given that the value of another random variable is known. 

That is: measure of how much entropy a random variable X has remaining if we have already learned the value of a second random variable Y. 

Here, we can try the following:

- conditional entropy of Class | Port 


In [13]:
port_ratios = data['embarked'].value_counts(normalize=True)
port_e = entropy(port_ratios)

# Ratios on Q
q_first_ratio = num_q_first / total_q
q_second_ratio = num_q_second / total_q
q_third_ratio = num_q_third / total_q

# Ratios on S
s_first_ratio = num_s_first / total_s
s_second_ratio = num_s_second / total_s
s_third_ratio = num_s_third / total_s


# Ratios on C
c_first_ratio = num_c_first / total_c
c_second_ratio = num_c_second / total_c
c_third_ratio = num_c_third / total_c

# Making series objects
q_class_ratio = pd.Series([q_first_ratio, q_second_ratio, q_third_ratio])
s_class_ratio = pd.Series([s_first_ratio, s_second_ratio, s_third_ratio])
c_class_ratio = pd.Series([c_first_ratio, c_second_ratio, c_third_ratio])

#Conditional Entropies Port-Class
q_class_cond_e = entropy(q_class_ratio)
s_class_cond_e = entropy(s_class_ratio)
c_class_cond_e = entropy(c_class_ratio)



In [14]:
''' num_q_first_f_y = len(data.loc[(data['embarked'] == 'Q') & (data['pclass'] == 1)])
num_q_first_m_y = 
num_q_first_f_a = 
num_q_first_m_a = 

num_adult_f = len(data.loc[(data['age'] == 1) & (data['sex'] == 0)])
'''

" num_q_first_f_y = len(data.loc[(data['embarked'] == 'Q') & (data['pclass'] == 1)])\nnum_q_first_m_y = \nnum_q_first_f_a = \nnum_q_first_m_a = \n\nnum_adult_f = len(data.loc[(data['age'] == 1) & (data['sex'] == 0)])\n"

## Tree 4: Class - Sex - Survival

In [15]:
class_ratios = data['pclass'].value_counts(normalize=True)
class_e = entropy(class_ratios)

# First Class
num_first_m = len(data.loc[(data['pclass'] == 1) & (data['sex'] == 1)])
num_first_f = len(data.loc[(data['pclass'] == 1) & (data['sex'] == 0)])
num_first = len(data.loc[(data['pclass'] == 1)])

ratio_first_m = num_first_m / num_first
ratio_first_f = num_first_f / num_first

# Second Class
num_sec_m = len(data.loc[(data['pclass'] == 2) & (data['sex'] == 1)])
num_sec_f = len(data.loc[(data['pclass'] == 2) & (data['sex'] == 0)])
num_sec = len(data.loc[(data['pclass'] == 2)])

ratio_sec_m = num_sec_m / num_sec
ratio_sec_f = num_sec_f / num_sec

# Third Class
num_third_m = len(data.loc[(data['pclass'] == 3) & (data['sex'] == 1)])
num_third_f = len(data.loc[(data['pclass'] == 3) & (data['sex'] == 0)])
num_third = len(data.loc[(data['pclass'] == 3)])

ratio_third_m = num_third_m / num_third
ratio_third_f = num_third_f / num_third

#Creating Series objects

first_ratio = pd.Series([ratio_first_m, ratio_first_f])
sec_ratio = pd.Series([ratio_sec_m, ratio_sec_f])
third_ratio = pd.Series([ratio_third_m, ratio_third_f])

first_cond_e = entropy(first_ratio)
sec_cond_e = entropy(sec_ratio)
third_cond_e = entropy(third_ratio)

# Getting weighted avg
ratio_first = num_first / total_pass
ratio_sec = num_sec / total_pass
ratio_third = num_third / total_pass

wt_avg_class_e = (ratio_first*(first_cond_e))+(ratio_sec*(sec_cond_e))+(ratio_third*(third_cond_e))

info_gain_class = info_gain(class_e, wt_avg_class_e)
print(info_gain_class)


0.5232460652146391


## Tree 5: Port - Class - Sex - Survival
Q,S,C - I, II, III - M,F
first_male_q / num_q
second_male_q / num_q
third_male_q / num_q

first_f_q / num_q
sec_f_q/num_q
third_f_q / num_q


In [19]:
port_ratios = data['embarked'].value_counts(normalize=True)
port_e = entropy(port_ratios)


# Males - First Class - Port Q
m_first_q =  len(data.loc[(data['embarked'] == 'Q') & (data['pclass'] == 1) & (data['sex'] == 1)])

y_m_first_q = len(data.loc[(data['embarked'] == 'Q') & (data['age'] == 0) & (data['pclass'] == 1) & (data['sex'] == 1)]) 
a_m_first_q = len(data.loc[(data['embarked'] == 'Q') & (data['age'] == 1) & (data['pclass'] == 1) & (data['sex'] == 1)])
o_m_first_q = len(data.loc[(data['embarked'] == 'Q') & (data['age'] == 2) & (data['pclass'] == 1) & (data['sex'] == 1)])

ymfq = y_m_first_q / m_first_q
amfq = a_m_first_q / m_first_q
omfq = o_m_first_q / m_first_q

# Males - Second Class - Port Q

#Generating Series object
m_first_q_ratio = pd.Series([ymfq, amfq, omfq])




## Tree 6: Port - Sex - Survival

In [21]:
port_ratios = data['embarked'].value_counts(normalize=True)
port_e = entropy(port_ratios)

'''# Total passengers based on PORT OF EMBARKATION
total_q = len(data.loc[data['embarked'] == 'Q'])
total_s = len(data.loc[data['embarked'] == 'S'])
total_c = len(data.loc[data['embarked'] == 'C'])'''

q_m = len(data.loc[(data['embarked'] == 'Q') & (data['sex'] == 1)])
q_f = len(data.loc[(data['embarked'] == 'Q') & (data['sex'] == 0)])

s_m = len(data.loc[(data['embarked'] == 'S') & (data['sex'] == 1)])
s_f = len(data.loc[(data['embarked'] == 'S') & (data['sex'] == 0)])

c_m = len(data.loc[(data['embarked'] == 'C') & (data['sex'] == 1)])
c_f = len(data.loc[(data['embarked'] == 'C') & (data['sex'] == 0)])

q_m_ratio = q_m / total_q
q_f_ratio = q_f / total_q

s_m_ratio = s_m / total_s
s_f_ratio = s_f / total_s

c_m_ratio = c_m / total_c
c_f_ratio = c_f / total_c

ratio_q = total_q / total_pass
ratio_s = total_s / total_pass
ratio_c = total_c / total_pass

#Generating series 
q_port = pd.Series([q_m_ratio, q_f_ratio])
s_port = pd.Series([s_m_ratio, s_f_ratio])
c_port = pd.Series([c_m_ratio, c_f_ratio])

#Getting conditional entropies
q_cond_e = entropy(q_port)
c_cond_e = entropy(c_port)
s_cond_e = entropy(s_port)

#Getting wt. avg
wt_avg_port_sex_e = (q_cond_e*ratio_q) + (s_cond_e*ratio_s) + (c_cond_e*ratio_c)

#information gain
info_gain_port = info_gain(port_e, wt_avg_port_sex_e)
print(info_gain_port)

0.22526026173172997
